### Importación de tabla y generación de tablas por actividad económica (PIB)

In [5]:
import pandas as pd
import chardet

with open('PIB_estatal_actividades_econ.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('PIB_estatal_actividades_econ.csv', skiprows=[0,1, 23, 24, 25], encoding=result['encoding'])\
        .set_index('Periodo')

# dividir el df en tres. Uno para cada tipo de actividad
text_to_delete = 'Cuentas nacionales > Producto interno bruto por entidad federativa, base 2013 > Por actividad económica y entidad federativa > Valores a precios constantes de 2013 >'
df.columns = [col.replace(text_to_delete, '').replace(' r1 / p1 / ', '') for col in df.columns.values]

# Create df with actividades primarias
df_prim_cols = [col for col in df.columns if 'primarias' in col]
df_prim = df[df_prim_cols]
text_to_delete = 'Actividades primarias > Total actividades primaria'
df_prim.columns = [col.replace(text_to_delete, '') for col in df_prim.columns.values]
df_prim.to_csv('data/PIB_primario.csv')

# Create df with actividades secundarias
df_sec_cols = [col for col in df.columns if 'secundarias' in col]
df_sec = df[df_sec_cols]
text_to_delete = 'Actividades secundarias > Total actividades secundarias'
df_sec.columns = [col.replace(text_to_delete, '') for col in df_sec.columns.values]
df_sec.to_csv('data/PIB_secundario.csv')

# Create df with actividades terciarias
df_ter_cols = [col for col in df.columns if 'terciarias' in col] 
df_ter = df[df_ter_cols]
text_to_delete = 'Actividades terciarias > Total actividades terciarias'
df_ter.columns = [col.replace(text_to_delete, '') for col in df_ter.columns.values]
df_ter.to_csv('data/PIB_terciario.csv')

### Ejemplo incompleto con API call a INEGI

In [4]:
# Ejemplo con API call
import pandas as pd
import requests
import json

# API call
# consultar: https://www.inegi.org.mx/servicios/api_indicadores.html
token = '5c520796-e0a7-f03b-0cdb-5101e2948d18'

url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/472079,472080,472081,472082,472083,472084,472085,472086,472087,472088/es/0700/true/BIE/2.0/{token}?type=json'
response= requests.get(url)
print(json.dumps(response.json(),indent=2))
if response.status_code==200:
    content= json.loads(response.content)
    Series=content['Series'][0]['OBSERVATIONS']   
    
    #Obtención de la lista de observaciones 
    Observaciones=[]
    for obs in Series:  Observaciones.append(float(obs['OBS_VALUE']));
    

    #Generación del promedio de la lista de observaciones 
    sum=0.0
    for i in range(0,len(Observaciones)): sum=sum+Observaciones[i];  

    resultado=sum/len(Observaciones);
    print(resultado)

{
  "Header": {
    "Name": "Datos compactos BIE",
    "Email": "atencion.usuarios@inegi.org.mx"
  },
  "Series": [
    {
      "INDICADOR": "472079",
      "FREQ": "3",
      "TOPIC": "102000430020002000700002",
      "UNIT": "317",
      "NOTE": "",
      "SOURCE": "17",
      "LASTUPDATE": "16/12/2019 01:19:03 p. m.",
      "OBSERVATIONS": [
        {
          "TIME_PERIOD": "2018",
          "OBS_VALUE": "3.12918e+006",
          "OBS_EXCEPTION": "",
          "OBS_STATUS": "1",
          "OBS_SOURCE": "17",
          "OBS_NOTE": null,
          "COBER_GEO": "00"
        }
      ]
    },
    {
      "INDICADOR": "472080",
      "FREQ": "3",
      "TOPIC": "102000430020002000700004",
      "UNIT": "317",
      "NOTE": "",
      "SOURCE": "17",
      "LASTUPDATE": "16/12/2019 01:19:03 p. m.",
      "OBSERVATIONS": [
        {
          "TIME_PERIOD": "2018",
          "OBS_VALUE": "3.12918e+006",
          "OBS_EXCEPTION": "",
          "OBS_STATUS": "1",
          "OBS_SOURCE": "17